In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import urllib



#Function

In [2]:


def process_csv(file_path):
    dtype_map = {
        "id": "int64",
        "Books": "string",
        "Days allowed to borrow": "string",
        "Customer ID": "Int64",
    }

    # Read raw, no parsing yet
    df = pd.read_csv(file_path, dtype=dtype_map, dayfirst=True)

    # Clean quotes from 'Book checkout' before parsing it as a date
    if 'Book checkout' in df.columns:
        df['Book checkout'] = df['Book checkout'].astype(str).str.replace('"', '').str.replace("'", '')

    # Now parse date columns safely
    df['Book checkout'] = pd.to_datetime(df['Book checkout'], errors='coerce', dayfirst=True)
    df['Book Returned'] = pd.to_datetime(df['Book Returned'], errors='coerce', dayfirst=True)

    df.dropna(how='all', inplace=True)

    # Fill NaNs sensibly
    df.fillna({
        **{col: -1 for col in df.select_dtypes(include='number').columns},
        **{col: 'Unknown' for col in df.select_dtypes(include=['object', 'string']).columns},
        **{col: pd.NaT for col in df.select_dtypes(include='datetime').columns}
    }, inplace=True)

    # Compute days between
    df['Days Between'] = (df['Book Returned'] - df['Book checkout']).dt.days
    df.loc[df['Days Between'] < 0, 'Book Returned'] = df.loc[df['Days Between'] < 0, 'Book checkout']
    df['Days Between'] = (df['Book Returned'] - df['Book checkout']).dt.days

    return df




In [3]:
def write_to_sql(df, table_name, server, database, username, password, if_exists='replace'):
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password}"
    )


#Output

In [4]:
if __name__ == "__main__":
    file_path = r"C:\Users\Admin\Desktop\QADEL5\data\03_Library Systembook.csv"
    cleaned_df = process_csv(file_path)
    print(cleaned_df) 

      Id                                     Books Book checkout  \
0    1.0                       Catcher in the Rye     2023-02-20   
1    2.0          Lord of the rings the two towers    2023-03-24   
2    3.0  Lord of the rings the return of the kind    2023-03-29   
3    4.0                                The hobbit    2023-04-02   
4    5.0                                     Dune     2023-04-02   
5    6.0                              Little Women    2023-04-02   
6    7.0                                        IT    2063-04-10   
7    8.0                                   Misery     2023-04-15   
8    9.0                                  Catch 22    2023-04-15   
9   10.0                              Animal Farm     2023-04-20   
10  11.0                                      1984    2023-04-23   
11  12.0                              Little Women    2023-04-02   
12  13.0                              East of Eden    2023-04-30   
13  14.0                   America Is in the Hea

In [5]:
write_to_sql(
        df=cleaned_df,
        table_name="LibraryBooks",
        server="localhost",
        database="LibraryDB",
        username="STUDENT14\Admin",
        password="",
        if_exists="replace"  # or 'append'
    )
